In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.utils import to_categorical

In [ ]:
def input_data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)


    return X_test, y_test, X_train, y_train


In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def evaluate_model(X_train, y_train, n_folds=5):
    accuracy = []
    data = []

    kfold = KFold(n_folds, shuffle=True, random_state=1)

    for train_ix, test_ix in kfold.split(X_train):
        model = create_model()

        train_X, train_y, test_X, test_y = X_train[train_ix], y_train[train_ix], X_train[test_ix], y_train[test_ix]

        data_fit = model.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=10, batch_size=32, verbose=0)
        
        _, acc = model.evaluate(test_X, test_y, verbose=0)
        
        accuracy.append(acc)
        data.append(data_fit)
    
    return accuracy, data

In [ ]:
def summarize_diagnostics(data):
    for i in range(len(data)):
        plt.subplot(2, 1, 1)
        plt.title('Cross Entropy Loss')
        plt.plot(data[i].history['loss'], color='red', label='train')
        plt.plot(data[i].history['val_loss'], color='orange', label='test')
        
        plt.subplot(2, 1, 2)
        plt.title('Classification Accuracy')
        plt.plot(data[i].history['accuracy'], color='blue', label='train')
        plt.plot(data[i].history['val_accuracy'], color='orange', label='test')
    plt.show()

In [ ]:
def summarize_performance(acc):
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (np.mean(acc) * 100, np.std(acc) * 100, len(acc)))
    plt.boxplot(acc)
    plt.show()

In [ ]:
def test(X_train, model):
    test_images = X_train[1:5]
    test_images = test_images.reshape(test_images.shape[0], 28, 28)

    for i, test_image in enumerate(test_images, start=1):
        org_image = test_image
        test_image = test_image.reshape(1, 28, 28, 1)
        prediction = model.predict_classes(test_image, verbose=0)

        print("Predicted digit: {}".format(prediction[0]))
        plt.subplot(220 + i)
        plt.axis('off')
        plt.title("Predicted digit: {}".format(prediction[0]))
        plt.imshow(org_image, cmap=plt.get_cmap('gray'))

    plt.show()

In [ ]:
def run():
    X_test, y_test, X_train, y_train = input_data()
    accuracy, data = evaluate_model(X_train, y_train)
    summarize_diagnostics(data)
    summarize_performance(accuracy)

    model = create_model()
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)
    test(X_train, model)

    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model.h5")
    print("Saved model to disk")

    img = cv2.imread('TestNumber.png')
    prediction = predict(img)
    print("Predicted Number:", prediction)

run()